<a href="https://colab.research.google.com/github/Yuhu-kth/ID2223/blob/main/cnn/feature_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 32.0 MB/s 
     |████████████████████████████████| 132 kB 77.4 MB/s 
     |████████████████████████████████| 182 kB 78.5 MB/s 
     |████████████████████████████████| 212 kB 77.0 MB/s 
     |████████████████████████████████| 127 kB 77.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 21.8 MB/s 
     |████████████████████████████████| 7.6 MB 44.3 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')	
import os
os.chdir('/content/gdrive/MyDrive/whisper/cnn')


Mounted at /content/gdrive


In [4]:
cd /content/drive/MyDrive/whisper/cnn

[Errno 2] No such file or directory: '/content/drive/MyDrive/whisper/cnn'
/content


In [5]:
ls

 common_voice/         __pycache__/                training_config.json
 env.yml               README.md                   training_pipline.py
 feature_pipeline.py   token.txt                   utils.py
 __init__.py          'training_config (1).json'


In [6]:
from datasets import load_dataset, DatasetDict, Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer
import sys
import huggingface_hub
import os

In [7]:
pip install "torchaudio<0.12"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 13.5 MB/s 
     |████████████████████████████████| 750.6 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.0+cu116
    Uninstalling torchaudio-0.13.0+cu116:
      Successfully uninstalled torchaudio-0.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.11.0 which is incompatible.


In [10]:
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
            It must be "yes" (the default), "no" or None (meaning
            an answer is required of the user).

    The "answer" return value is True for "yes" or False for "no".
    
    from: https://stackoverflow.com/questions/3041986/apt-command-line-interface-like-yes-no-input
    """
    valid = {"yes": True, "y": True, "ye": True, "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == "":
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' " "(or 'y' or 'n').\n")
            
def load_common_voice(path=None, save_path=None):
    """function that loads or downloads and edits

    Args:
        path (string, optional): path to the dataset to load. Defaults to None. (directory)
        save_path (string, optional): path where to save the loaded/downloaded dataset. Defaults to None. (directory)

    Returns:
        datasets.DatasetDict: common voice dataset
    """
    
    # if the save path already exists, ask the user whether they want to overwrite it
    if save_path and os.path.exists(save_path):
        if not query_yes_no(f"{save_path} already exists and will be overwritten. Continue?"):
            return
        
    # if the save path is same to load path (and they exist), we may want to load it instead
    if path == save_path and path:
        if query_yes_no(f"{save_path} already exists. Do you want to load it instead?"):
            return DatasetDict.load_from_disk(save_path)
    
    print("Dataset loading started")
    
    if path:
        print(f"Loading dataset from {path}...")
        return DatasetDict.load_from_disk(path)

    print("Loading dataset from huggingface...")
    common_voice = DatasetDict()

    common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
    common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)
    
    print("Raw dataset loaded.")

    common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
    
    feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
    tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")
    
    def prepare_dataset(batch):
        # load and resample audio data from 48 to 16kHz
        audio = batch["audio"]

        # compute log-Mel input features from input audio array 
        batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

        # encode target text to label ids 
        batch["labels"] = tokenizer(batch["sentence"]).input_ids
        return batch

    print("Mapping the dataset...")
    common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)
    
    print("Dataset ready for training.")
    
    if SAVE_DATASET_PATH:
        print(f"Saving dataset to {save_path}...")
        common_voice.save_to_disk(save_path)
        
        return common_voice
        
        
if __name__ == "__main__":
    print("Feature pipeline started...")
    
    # path to read the common voice dataset from (directory)
    # if None, common voice will be downloaded from huggingface
    LOAD_DATASET_PATH = None

    # path to save the common voice dataset to after downloading (directory)
    # if None, it will not be saved
    SAVE_DATASET_PATH = "common_voice"

    with open("token.txt", mode='r') as f:
        token = f.read()
        
    huggingface_hub.login(token=token, add_to_git_credential=True)
    
    if not LOAD_DATASET_PATH and not SAVE_DATASET_PATH:
        print("Error: no paths were specified to load nor save, so this code will have no effect. Aborting.")
        sys.exit()

    load_common_voice(path=LOAD_DATASET_PATH, save_path=SAVE_DATASET_PATH)

Feature pipeline started...
Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful
common_voice already exists and will be overwritten. Continue? [Y/n] Y
Dataset loading started
Loading dataset from huggingface...


Raw dataset loaded.
Mapping the dataset...
 

Dataset ready for training.
Saving dataset to common_voice...


Saving the dataset (0/24 shards):   0%|          | 0/12360 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/5069 [00:00<?, ? examples/s]